# SSEN Network Constraints Analysis

This notebook analyzes the SSEN (Scottish and Southern Electricity Networks) low-voltage feeder metadata to extract physical constraints for anomaly detection verification.

**Purpose**: Define realistic physical constraints that the verifier agent will use to validate energy consumption scenarios.
